In [ ]:
# Configurations for playbook
CURR_DIRECTORY = '' # Directory to look for emails

In [ ]:
import sys
!{sys.executable} -m pip install python-magic
!{sys.executable} -m pip install opencv-python

In [ ]:
import os
import email
import cv2
import magic
# Please run 'brew install libmagic' in terminal.

In [ ]:
try:
   os.makedirs('emls_to_parse/raw_emls/')
except OSError as err:
   print(err)

listOfEmls = os.listdir(CURR_DIRECTORY + 'emls_to_parse/raw_emls')
# Add the .eml files to this path above.

In [ ]:
def mime_flagchecks(file_path):
    return 'image' in magic.from_file(file_path, mime=True)

In [ ]:
def get_qr_code(file_path, eml):
    image = cv2.imread(file_path)
    detector = cv2.QRCodeDetector()

    path1 = f'{CURR_DIRECTORY}/emls_to_parse/eml_qr_urls.csv'
    path2 = f'{CURR_DIRECTORY}/emls_to_parse/eml_qr_error.csv'

    with open(path1, 'a') as f_qr_urls, open(path2, mode="a") as f_errored:
        try:
            url, vertices_array, binary_qrcode = detector.detectAndDecode(image)
            if vertices_array is not None:
                f_qr_urls.write(eml + ", " + url + '\n')
            else:
                f_errored.write(eml + ", " + "Errored at vertices_array" + '\n')
        except:
            f_errored.write(eml + ", " + "Errored at CV detectAndDecode" + '\n')

    f_errored.close()
    f_qr_urls.close()

In [ ]:
for eml in listOfEmls:

    current_file = os.path.join(CURR_DIRECTORY, 'emls_to_parse/raw_emls/', eml)

    with open(current_file, 'rb') as f:

        msg = email.message_from_binary_file(f)

        for part in msg.walk():
            fileName = part.get_filename()

            content = part.get_payload(decode=True)
            if content:
                extraction_path = CURR_DIRECTORY + 'emls_to_parse/tmp_image'
                with open(extraction_path, 'wb') as save_attachment:
                    save_attachment.write(content)
                save_attachment.close()

                flag = mime_flagchecks(extraction_path)

                if flag:
                    qr_url = get_qr_code(extraction_path, eml)

    f.close()